In [1]:
# Внимание!!! Важно, что бы файлы с данными и исполняемый файл находились в одной папке, 
# тогда пути к тестовым и тренировочным наборам будут содержать только имена файлов.
# 
# В пути к тренировочным и тестовым данным запрежается использовать абсалютную адресацию, 
# то есть адресацию, в которой присутствуют имена папок. Путь должен содержать только имя файла.
#
# Напоминание: под моделью машинного обучения понимаются все действия с исходными данными, 
# которые необходимо произвести, что бы сопоставить признаки целевому значению.

### Область работы 1 (библиотеки)

In [2]:
# Данный блок в области 1 выполняется преподавателем
# 
# данный блок предназначен только для подключения необходимых библиотек
# запрещается подключать библиотеки в других блоках
# запрещается скрывать предупреждения системы
# установка дополнительных библиотек размещается прямо здесь (обязательно закоментированы)
# pip install

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time

from IPython.display import display

from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures, StandardScaler, OneHotEncoder, RobustScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_validate, KFold, GridSearchCV, StratifiedShuffleSplit
from sklearn.neighbors import KNeighborsRegressor 

from sklearn.linear_model import ElasticNet, Ridge, Lasso, Lars
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline, make_pipeline

# pip install category-encoders

import category_encoders as ce

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer

from sklearn.metrics import classification_report, mean_absolute_percentage_error

### Область работы 2 (выполнение лучшей модели)

In [5]:
# Данный блок(и) в области 2 выполняется преподавателем
#
# В области находится одна, единственная, итоговая модель машинного обучения с однозначными, 
# зафиксированными параметрами
#
# В данной области категорически запрещается искать, выбирать, улучшать, оптимизировать, 
# тюниговать и т.д. модель машинного обучения

In [15]:
# Путь к тренировочному набору
path_train = 'train.csv' # содержит только имя файла, без имен папок
# Путь к тестовому набору
path_test  = 'test.csv' # содержит только имя файла, без имен папок

In [16]:
# Блок(и) обучения и поверки модели

In [17]:
diamonds_train = pd.read_csv(path_train)
diamonds_test = pd.read_csv(path_test)

In [18]:
diamonds_train.drop_duplicates(inplace = True)

In [19]:
diamonds_train = diamonds_train.drop(['symmetry', 'polish', 'depth_percent'], axis = 1)
diamonds_test = diamonds_test.drop(['symmetry', 'polish', 'depth_percent'], axis = 1)

In [20]:
y = np.array(diamonds_train.total_sales_price)
X = diamonds_train.drop(columns=['total_sales_price'])

X_test = diamonds_test

In [21]:
cat_features = ['cut']
axis_features = ['meas_length','meas_width','meas_depth']
num_features = ['size','table_percent']

In [28]:
axis_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(missing_values=0, n_neighbors=4, weights="uniform")),
    ('polynom', PolynomialFeatures(2,include_bias=False)),
    ('scaler', MinMaxScaler())])

clarity_map = [{
    'col':'clarity',
    'mapping':{'IF':10, 'VVS1':9, 'VVS2':8, 'VS1':7, 'VS2':6, 
               'SI1':5, 'SI2':4, 'I1':3, 'I2':2, 'I3':1}
    }]

clarity_transformer = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping=clarity_map)),
    ('scaler', StandardScaler())
    ])

color_map = [{
    'col':'color',
    'mapping':{'D':10, 'E':9, 'F':8, 'G':7, 'H':6, 'I':5, 
               'J':4, 'K':3, 'L':2, 'M':1}
    }]

color_transformer = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping=color_map)),
    ('scaler', RobustScaler())
    ])
    
CT = ColumnTransformer([
        ("pol_std", axis_transformer, axis_features),
        ("num", MinMaxScaler(), num_features),
        ("cat", OneHotEncoder(), cat_features),
        ("ordinal_clarity_map", clarity_transformer, ['clarity']),
        ("ordinal_color_map", color_transformer, ['color'])
        ])

display(CT)

ColumnTransformer(transformers=[('pol_std',
                                 Pipeline(steps=[('imputer',
                                                  KNNImputer(missing_values=0,
                                                             n_neighbors=4)),
                                                 ('polynom',
                                                  PolynomialFeatures(include_bias=False)),
                                                 ('scaler', MinMaxScaler())]),
                                 ['meas_length', 'meas_width', 'meas_depth']),
                                ('num', MinMaxScaler(),
                                 ['size', 'table_percent']),
                                ('cat', OneHotEncoder(), ['cut']),
                                ('ordinal_clarity_map',
                                 Pipeline(st...
                                                  OrdinalEncoder(mapping=[{'col': 'clarity',
                                                                           'data_type': dtype('O'),
                                                                           'mapping': IF      10
VVS1     9
VVS2     8
VS1      7
VS2      6
SI1      5
SI2      4
I1       3
I2       2
I3       1
dtype: int64}])),
                                                 ('scaler', StandardScaler())]),
                                 ['clarity']),
                                ('ordinal_color_map',
                                 Pipeline(steps=[('ce',
                                                  OrdinalEncoder(mapping=[{'col': 'color',
                                                                           'data_type': dtype('O'),
                                                                           'mapping': D    10
E     9
F     8
G     7
H     6
I     5
J     4
K     3
L     2
M     1
dtype: int64}])),
                                                 ('scaler', RobustScaler())]),
                                 ['color'])])

In [29]:
pipe_best = Pipeline([('preprocessing', CT),
                 ('regressor', KNeighborsRegressor(n_neighbors = 15, p = 1, weights='distance'))])
pipe_best.fit(X, y)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('pol_std',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer(missing_values=0,
                                                                              n_neighbors=4)),
                                                                  ('polynom',
                                                                   PolynomialFeatures(include_bias=False)),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['meas_length', 'meas_width',
                                                   'meas_depth']),
                                                 ('num', MinMaxScaler(),
                                                  ['size', 'table_percent']),
                                                 ('cat', OneHotEncoder(),
                                                  ['cut']),
                                                 (...
VS1      7
VS2      6
SI1      5
SI2      4
I1       3
I2       2
I3       1
dtype: int64}])),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['clarity']),
                                                 ('ordinal_color_map',
                                                  Pipeline(steps=[('ce',
                                                                   OrdinalEncoder(mapping=[{'col': 'color',
                                                                                            'data_type': dtype('O'),
                                                                                            'mapping': D    10
E     9
F     8
G     7
H     6
I     5
J     4
K     3
L     2
M     1
dtype: int64}])),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  ['color'])])),
                ('regressor',
                 KNeighborsRegressor(n_neighbors=15, p=1, weights='distance'))])

In [30]:
# Блок предсказания с использованием тестового набора

In [31]:
y_predict = pipe_best.predict(X_test)

In [32]:
# Название вектора предсказанных значений  y_predict полученого на основании тестового набора
y_predict

array([2090.43079104,  934.62211081, 1791.03336136, ..., 1228.15035744,
       2993.00339051, 1675.67760028])